## Loading Packages

In [2]:
import numpy as np
import pandas as pd
import re
import warnings

import seaborn as sn
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

## Loading Datasets

In [7]:
#games = pd.read_csv('../input/nba-games/games.csv')
# details = pd.read_csv('../input/nba-games/games_details.csv')
# teams = pd.read_csv('../input/nba-games/teams.csv')
# players = pd.read_csv('../input/nba-games/players.csv')
# ranking = pd.read_csv('../input/nba-games/ranking.csv')
team_record = pd.read_csv("Player Data Updated/Team_Records.csv")

## Data Overview

## Data Cleaning


## Data Filling

## Exploratory Data Analysis


## Hypothesis Builiding

## Feature Engineering & Selection


## ML Model Selection

## ML Fit & Training

## ML Validation Testing

## ML Prediction and Analysis